# Download data
-------------------------

# Imports

In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
import os
import pandas as pd
from pathlib import Path
import sys
sys.path.insert(0, '..')

from src.utils.utils import save_csv, create_directory, download_file_from_url, unzip_file

In [47]:
# Find .env automagically by walking up directories until it's found, then load up the .env entries as environment variables
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())
project_dir = os.path.join(os.path.dirname('.env'), os.pardir)

DATA_DIR = os.path.join(project_dir, os.environ.get("DATA_DIR"))
EXTERNAL_DATA_DIR = os.path.join(project_dir, os.environ.get("EXTERNAL_DATA_DIR"))
RAW_DATA_DIR = os.path.join(project_dir, os.environ.get("RAW_DATA_DIR"))
PROCESSED_DATA_DIR = os.path.join(project_dir, os.environ.get("PROCESSED_DATA_DIR"))

In [48]:
RAW_ZIP_DATA_DIR = RAW_DATA_DIR + 'zip/'
RAW_CSV_DATA_DIR = RAW_DATA_DIR + 'csv/'

# Part 1 - Load data

In [49]:
file_path = os.path.join(RAW_DATA_DIR, 'master_file_list.txt')
df = pd.read_csv(file_path, sep=' ', header=None, names=['something', 'id', 'url'])
df

C:\Users\louis\AppData\Local\Temp\ipykernel_31148\1527058827.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=' ', header=None, names=['something', 'id', 'url'])


,something,id,url
0,150383,297a16b493de7cf6ca809a7cc31d0b93,http://data.gdeltproject.org/gdeltv2/201502182...
1,318084,bb27f78ba45f69a17ea6ed7755e9f8ff,http://data.gdeltproject.org/gdeltv2/201502182...
2,10768507,ea8dde0beb0ba98810a92db068c0ce99,http://data.gdeltproject.org/gdeltv2/201502182...
3,149211,2a91041d7e72b0fc6a629e2ff867b240,http://data.gdeltproject.org/gdeltv2/201502182...
4,339037,dec3f427076b716a8112b9086c342523,http://data.gdeltproject.org/gdeltv2/201502182...
...,...,...,...
814786,108304,81dbf12b7dae6baa0cac2d2f9d5ef42c,http://data.gdeltproject.org/gdeltv2/202301131...
814787,5336143,22c9f0e26855dcbd48a0f3c7a3d91906,http://data.gdeltproject.org/gdeltv2/202301131...
814788,77342,0c14e3c325fc7a064e851e9fe0f05734,http://data.gdeltproject.org/gdeltv2/202301131...
814789,128699,669f07e39c97541f908c5b3aaad00d43,http://data.gdeltproject.org/gdeltv2/202301131...


In [83]:
urls_to_process = df['url'].tolist()

n_urls = 5
urls_to_process = urls_to_process[:n_urls]

for row, url in enumerate(urls_to_process):
    zip_file_path = os.path.join(RAW_ZIP_DATA_DIR, Path(url).name)
    csv_file_path = os.path.join(RAW_CSV_DATA_DIR, Path(url).stem)
 
    # Download data from remote url to data/raw/zip
    create_directory(RAW_ZIP_DATA_DIR)
    if not os.path.isfile(zip_file_path):
        download_file_from_url(remote_url=url, local_dir=RAW_ZIP_DATA_DIR, verbose=True)
    else:
        print(f"[{row}/{len(urls_to_process)}] File '{zip_file_path}' already exists.")


    # Unzip file within data/raw/url to data/raw/csv
    create_directory(RAW_CSV_DATA_DIR)
    if not os.path.isfile(csv_file_path):
        unzip_file(zip_file_path, RAW_CSV_DATA_DIR, verbose=True)
    else:
        print(f"[{row}/{len(urls_to_process)}] File '{csv_file_path}' already exists.")

[0/5] File '..\./data/raw/zip/20150218230000.export.CSV.zip' already exists.
[0/5] File '..\./data/raw/csv/20150218230000.export.CSV' already exists.
[1/5] File '..\./data/raw/zip/20150218230000.mentions.CSV.zip' already exists.
[1/5] File '..\./data/raw/csv/20150218230000.mentions.CSV' already exists.
[2/5] File '..\./data/raw/zip/20150218230000.gkg.csv.zip' already exists.
[2/5] File '..\./data/raw/csv/20150218230000.gkg.csv' already exists.
[3/5] File '..\./data/raw/zip/20150218231500.export.CSV.zip' already exists.
[3/5] File '..\./data/raw/csv/20150218231500.export.CSV' already exists.
[4/5] File '..\./data/raw/zip/20150218231500.mentions.CSV.zip' already exists.
[4/5] File '..\./data/raw/csv/20150218231500.mentions.CSV' already exists.


In [78]:
csv_files = [f for f in os.listdir(RAW_CSV_DATA_DIR)]
print(f"{len(csv_files)} csv file found within {RAW_CSV_DATA_DIR}")
csv_files

2 csv file found within ..\./data/raw/csv/


['20150218230000.export.CSV', '20150218230000.mentions.CSV']

In [52]:
# df_csv = pd.DataFrame()
# for csv_file in csv_files:
#     csv_path = RAW_CSV_DATA_DIR+csv_file
#     print(csv_path)
#     df_tmp = pd.read_csv(csv_path, sep='\t', header=None)
#     df_csv = pd.concat([df_csv, df_tmp], ignore_index=True)
# df_csv

In [53]:
# ssh lbrejon-21@tp-1a252-20.enst.fr
# ssh lbrejon-21@tp-1a252-21.enst.fr

In [54]:
# cd /home/users/lbrejon-21

In [55]:
# ssh lbrejon-21@ssh.enst.fr 
# ssh ubuntu@137.194.211.146
# ssh tp-hadoop-55